# Part 1

In [ ]:
with open("input.txt") as input:
  lines = input.readlines()

lines = [x.split(" ") for x in lines]

all_lights, all_buttons, all_joltage = [], [], []
for line in lines:
  all_lights.append(tuple(map(lambda x: True if x == "#" else False, line[0][1:len(line[0])-1])))
  all_buttons.append(list(map(lambda x: list(map(int,x[1:-1].split(","))),line[1:-1])))
  # all_joltage.append(list(map(int,line[-1].strip()[1:-1].split(",")))) # Not needed for part 1

def toggle_lights(lights, switches):
  lights = list(lights)
  for switch in switches: lights[switch] = not lights[switch]
  return tuple(lights)

part1 = 0

for i in range(len(lines)):
  correct_lights = all_lights[i]
  buttons = all_buttons[i]
  counter = 0 
  found_correct_lights = False
  lights = [[False] * len(correct_lights)]
  while not found_correct_lights:
    counter += 1
    temp_lights = set()
    for button in buttons:
      for light in lights:
        temp_light = toggle_lights(light,button)
        found_correct_lights = temp_light == correct_lights
        if found_correct_lights:
          break
        else:
          temp_lights.add(temp_light)
      if found_correct_lights: break
    lights = list(temp_lights)
  part1 += counter

print("Part 1 Answer",part1)

Part 1 Answer 486


# Part 2

In [ ]:
import numpy as np
from scipy.optimize import linprog
import pulp

with open("input.txt") as input:
  lines = input.readlines()

lines = [x.split(" ") for x in lines]

all_lights, all_buttons, all_joltage = [], [], []
for line in lines:
  # all_lights.append(tuple(map(lambda x: True if x == "#" else False, line[0][1:len(line[0])-1]))) # Not needed for part 2
  all_buttons.append(list(map(lambda x: list(map(int,x[1:-1].split(","))),line[1:-1])))
  all_joltage.append(list(map(int,line[-1].strip()[1:-1].split(","))))

part2 = 0

for i in range(len(lines)):
  target_joltages = all_joltage[i]
  buttons = all_buttons[i]
  # So, we want to solve the equation while minimizing the sum of all variables, i.e the number of button presses. 
  # Can be done using algorithm that goes row by row and max one vairable and set rest to 0. This will yield one correct solution really fast... Super easy to do by hand! More difficult to generalize.

  # Start by adding function to minimize
  c = np.ones(len(buttons)) 
  
  # Create the matrix representing all different types of buttons
  A = []
  for i in range(len(target_joltages)):
    row = [0] * len(buttons)
    for j in range(len(buttons)):
      if i in buttons[j]:
        row[j] = 1
    A.append(row)
  B = np.array(target_joltages)
  A = np.array(A)

  # using optimizer to solve for x in Ax = B with bounderies for x. 
  n = len(buttons)
  m = len(target_joltages)
  prob = pulp.LpProblem("MinimizeButtons",pulp.LpMinimize)
  x = [pulp.LpVariable(f"x{i}", lowBound=0, cat="Integer") for i in range(n)]
  prob += pulp.lpSum(c[j] * x[j] for j in range(n))
  for i in range(m):
      prob += pulp.lpSum(A[i][j] * x[j] for j in range(n)) == B[i]
  prob.solve(pulp.PULP_CBC_CMD(msg=0))
  solution_values = [var.varValue for var in x]
  # ------------------------------------------------------------------

  part2 += int(sum(solution_values))

print("Part 2 Answer",part2)

Part 2 Answer 17820
